# BSS-AUS: Basketball Statistic System (AUS)

This system allows to **incrementally** build a database of _stint_ (advance) statistics for each game and each team.

A **stint** is a lineup of players who play together in different interval periods across the game. The system will build the stints for each team from the play-by-play data and compute various statistics.

The data comes as a raw JSON file:

https://fibalivestats.dcd.shared.geniussports.com/data/2087737/data.json

In [1]:
# Let's first load all required packages...
import os
import pandas as pd

from config import *
import bball_stats
import tools

## 1. Compute stat tables for various games


## 1. Define games to scrape and saved data

First, setup the games we want to scrape and compute, as well as the existing data stored in file to append to.

In [2]:
# games to be computed
# format: (game id, game no for team 1, game no for team 2)
games = [(1999318, 1, 1), (2087737, 2, 2), (2090351, 3, 3)]

reload = False
file_stats_df = os.path.join(DATA_DIR, "stats_df.pkl")
file_games_df = os.path.join(DATA_DIR, "games_df.pkl")

## 2. Compute stat and game tables

Now, let us run the system that scrapes the games' data, compute stats and game info, and adds them to the initial tables of stats and games.

In [3]:
init_stats_df = None
game_df = None
if os.path.exists(file_stats_df) and not reload:
    # load the stat dataframe already stored as a file
    print(f"Loading initial stats df: {file_stats_df}")
    init_stats_df = pd.read_pickle(file_stats_df)
    game_df = pd.read_pickle(file_games_df)
    existing_games = init_stats_df.game_id.unique()
else:
    existing_games = []

stats_dfs = [init_stats_df] if init_stats_df is not None else []
games_data = []
for game in games:
    if game[0] in existing_games:
        print(f"Game {game[0]} is already in table")
        continue
    print(f"Computing game {game[0]}...")

    # now compute the actual stats for the game
    df, team1, team2 = bball_stats.build_game_stints_stats_df(game[0])
    print(f"\t .... done: {team1[0]} ({team1[1]}) vs {team2[0]} ({team2[1]})")

    # fill game
    df.insert(0, 'game_id', game[0])
    df.insert(3, 'game_no1', game[1])
    df.insert(4, 'game_no2', game[2])
    stats_dfs.append(df)
    games_data.append({'game_id': game[0],
                       "team1": team1[0], "team2": team2[0],
                       "s1": team1[1], "s2": team2[1],
                       "game_no1": game[1], "game_no2": game[2]})

# put all dfs together into a single dataframe
stats_df = pd.concat(stats_dfs)
stats_df.reset_index(inplace=True, drop=True)
stats_df.sample(5)

if game_df is not None:
    games_df = pd.concat([game_df, pd.DataFrame(games_data)])
    games_df.reset_index(inplace=True, drop=True)
else:
    games_df = pd.DataFrame(games_data)

Loading initial stats df: data/stats_df.pkl
Game 1999318 is already in table
Game 2087737 is already in table
Game 2090351 is already in table


If we want we can do some sanity checks, before saving to disk:

In [4]:
# Check the two df computed
games_df
# stats_df
# stats_df.loc[4]

,game_id,team1,team2,s1,s2,game_no1,game_no2
0,1999318,Melbourne United,New Zealand Breakers,90,84,1,1
1,2087737,Melbourne United,Tasmania JackJumpers,73,76,2,2
2,2090351,Tasmania JackJumpers,Sydney Kings,86,90,3,3


In [5]:
# (ortg, drtg) should mirror (drtg_opp, ortg)
stats_df.iloc[41][['poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]]

poss         10.88
ortg         73.53
drtg        101.21
poss_opp      9.88
ortg_opp    101.21
drtg_opp     73.53
Name: 41, dtype: object

## 3. Save stats and games to files

We now save the full dataframes (stats and games) in various formats: binary (pickle), csv, and Excel.

This will allows us to re-load that data later to add more games to it quicker.

In [6]:
import os

stats_df.to_pickle(os.path.join(DATA_DIR, "stats_df.pkl"))
games_df.to_pickle(os.path.join(DATA_DIR, "games_df.pkl"))

stats_df.to_csv(os.path.join(DATA_DIR, "stats_df.csv"))
games_df.to_csv(os.path.join(DATA_DIR, "games_df.csv"))

stats_df.to_excel(os.path.join(DATA_DIR, "stats_df.xlsx"))
games_df.to_excel(os.path.join(DATA_DIR, "games_df.xlsx"))


### 3. Inspection & analysis

We use [dtale](https://pypi.org/project/dtale/) package for this.

In [10]:
import dtale

dtale.show(stats_df[['tno', 'stint', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]])

import dtale

In [12]:
dtale.show(games_df)

In [13]:
dtale.show(stats_df)